In [1]:
import backtrader as bt
import yfinance as yf
import pyfolio as pf
import warnings
warnings.filterwarnings('ignore')

C:\Users\juanc\anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
df = yf.download("BTC-USD")

[*********************100%***********************]  1 of 1 completed


In [6]:
class Estrategia(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    params = dict(fast = 50,
                  slow = 200)
        
    def __init__(self):
        smas = bt.indicators.SMA(self.p.fast)
        smal = bt.indicators.SMA(self.p.slow)
        self.crossover = bt.ind.CrossOver(smas, smal)
        
    def next(self):
        if not self.position:      # Si no hay ninguna operacion abierta
            if self.crossover > 0: # si la rapida cruza hacia arriba la lenta
                self.buy()         # Se entra en LONG
            if self.crossover < 0: # si la rapida cruza hacia abajo la lenta
                self.sell()        # Se entra en SHORT

        elif self.crossover != 0:  # Si hay una operacion abierta y el cruce es hacia abajo 
            self.close() 

In [7]:
cerebro = bt.Cerebro()
    
cerebro.broker.setcash(100000.0)
data = bt.feeds.PandasData(dataname = df)
cerebro.adddata(data)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

In [ ]:
cerebro.addstrategy(Estrategia)

cerebro.run()

In [ ]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
cerebro.plot()